In [5]:
## 首先生成随机序列，并保存为FSM
# 第三步：状态机筛选
import random
import numpy as np

uuid = "c1d0f855d0f841d00c1d0000000000c1"

def uuid_to_binary(uuid_str):
    """将UUID字符串转换为128位二进制，并分成4个32位"""
    # 移除所有破折号并转换为二进制
    uuid_int = int(uuid_str, 16)
    uuid_bin = format(uuid_int, '0128b')
    # 分成4个32位
    return [uuid_bin[i:i+32] for i in range(0, 128, 32)]

## 获取状态机初始状态
def get_remaining_bits(uuid_str):
    """从UUID的四个部分中提取最后一位并拼接成4位二进制"""
    # 使用之前的函数将UUID转换为二进制并分段
    uuid_parts = uuid_to_binary(uuid_str)
    
    # 从每个32位部分取最后一位（即第31位，因为索引从0开始）
    remaining_bits = ''.join(part[31] for part in uuid_parts)
    
    return remaining_bits

four_bits = get_remaining_bits(uuid)

## 定义混沌状态机
class NonlinearFSM:
    def __init__(self, modulus=16):
        """
        初始化FSM，使用混沌映射的思想
        :param modulus: 模数，用于限制状态的范围
        """
        self.modulus = modulus
        # 使用多个无理数作为混沌参数
        self.phi = (1 + 5 ** 0.5) / 2  # 黄金分割比
        self.e = 2.718281828459045     # 自然对数e
        self.pi = 3.141592653589793    # 圆周率π
        self.sqrt2 = 2 ** 0.5          # 根号2
        
    def next_state(self, current_state):
        """
        使用更复杂的混沌映射计算下一个状态
        状态驱动越复杂，越安全，因为这样公布选取了那些行和列，然后
        """
        # 归一化当前状态到[0,1]区间
        x = current_state / self.modulus
        
        # 使用多个混沌映射的组合
        logistic = 4 * x * (1 - x)                    # Logistic映射
        tent = min(2 * x, 2 * (1 - x))               # Tent映射
        sine = abs(np.sin(self.pi * x))              # Sine映射
        cubic = abs((3 * x ** 3 - 2 * x) % 1)        # 三次映射
        
        # 混合多个映射结果
        mixed = (logistic * self.phi + 
                tent * self.e + 
                sine * self.pi + 
                cubic * self.sqrt2) % 1
        
        # 添加非线性扰动
        disturbed = (mixed + x * self.phi) % 1
        
        # 将结果映射回原始范围
        state_prime = int(disturbed * self.modulus)
        
        # 确保不会停留在同一个状态
        if state_prime == current_state:
            state_prime = (state_prime + int(self.phi * 10)) % self.modulus
            
        return state_prime

    def iterate(self, initial_state, iterations):
        """
        根据初始状态和迭代次数生成结果
        """
        current_state = initial_state
        results = []
        for _ in range(iterations):
            current_state = self.next_state(current_state)
            results.append(current_state)
        return results

## 计算筛选器
fsm = NonlinearFSM()
# 指定初始状态和迭代次数
initial_state = int(four_bits, 2) 
results = fsm.iterate(initial_state, 1024)
print(results)

import struct
with open('./megabitrand.bin', 'wb') as f:
    for i in range(0, len(results), 2):
        # 将每两个数字组合为一个字节
        byte_value = (results[i] << 4) | results[i+1]
        f.write(struct.pack('B', byte_value))
print("OK")

[8, 13, 7, 4, 15, 11, 10, 2, 13, 7, 4, 15, 11, 10, 2, 13, 7, 4, 15, 11, 10, 2, 13, 7, 4, 15, 11, 10, 2, 13, 7, 4, 15, 11, 10, 2, 13, 7, 4, 15, 11, 10, 2, 13, 7, 4, 15, 11, 10, 2, 13, 7, 4, 15, 11, 10, 2, 13, 7, 4, 15, 11, 10, 2, 13, 7, 4, 15, 11, 10, 2, 13, 7, 4, 15, 11, 10, 2, 13, 7, 4, 15, 11, 10, 2, 13, 7, 4, 15, 11, 10, 2, 13, 7, 4, 15, 11, 10, 2, 13, 7, 4, 15, 11, 10, 2, 13, 7, 4, 15, 11, 10, 2, 13, 7, 4, 15, 11, 10, 2, 13, 7, 4, 15, 11, 10, 2, 13, 7, 4, 15, 11, 10, 2, 13, 7, 4, 15, 11, 10, 2, 13, 7, 4, 15, 11, 10, 2, 13, 7, 4, 15, 11, 10, 2, 13, 7, 4, 15, 11, 10, 2, 13, 7, 4, 15, 11, 10, 2, 13, 7, 4, 15, 11, 10, 2, 13, 7, 4, 15, 11, 10, 2, 13, 7, 4, 15, 11, 10, 2, 13, 7, 4, 15, 11, 10, 2, 13, 7, 4, 15, 11, 10, 2, 13, 7, 4, 15, 11, 10, 2, 13, 7, 4, 15, 11, 10, 2, 13, 7, 4, 15, 11, 10, 2, 13, 7, 4, 15, 11, 10, 2, 13, 7, 4, 15, 11, 10, 2, 13, 7, 4, 15, 11, 10, 2, 13, 7, 4, 15, 11, 10, 2, 13, 7, 4, 15, 11, 10, 2, 13, 7, 4, 15, 11, 10, 2, 13, 7, 4, 15, 11, 10, 2, 13, 7, 4, 15, 11, 10,

In [3]:
import random
import numpy as np

uuid = "c1d0f855d0f841d00c1d0000000000c1"

def uuid_to_binary(uuid_str):
    """将UUID字符串转换为128位二进制，并分成4个32位"""
    uuid_int = int(uuid_str, 16)
    uuid_bin = format(uuid_int, '0128b')
    return [uuid_bin[i:i+32] for i in range(0, 128, 32)]

def get_remaining_bits(uuid_str):
    """从UUID的四个部分中提取最后一位并拼接成4位二进制"""
    uuid_parts = uuid_to_binary(uuid_str)
    remaining_bits = ''.join(part[31] for part in uuid_parts)
    return remaining_bits

four_bits = get_remaining_bits(uuid)

class NonlinearFSM:
    def __init__(self):
        self.phi = (1 + 5 ** 0.5) / 2
        self.e = 2.718281828459045
        self.pi = 3.141592653589793
        self.sqrt2 = 2 ** 0.5

    def next_state(self, current_state):
        # 保持状态值为小数类型
        x = current_state
        logistic = 4 * x * (1 - x)
        # tent = min(2 * x, 2 * (1 - x))
        tent = min(1.9 * x, 1.9 * (1 - x))
        sine = abs(np.sin(self.pi * x))
        cubic = abs((3 * x ** 3 - 2 * x) % 1)
        mixed = (logistic * self.phi + tent * self.e + sine * self.pi + cubic * self.sqrt2) % 1
        # mixed = (logistic + tent + sine + cubic) % 1
        disturbed = (mixed + x * self.phi) % 1
        return disturbed  # 返回一个小数状态值
        # return mixed

    def iterate_and_find_period(self, initial_state, max_iterations=2048):
        current_state = initial_state
        seen_states = {}
        results = []
        for iteration in range(max_iterations):
            # 使用当前状态的小数形式
            if current_state in seen_states:
                period_length = iteration - seen_states[current_state]
                return results, period_length
            seen_states[current_state] = iteration
            current_state = self.next_state(current_state)
            results.append(current_state)
        return results, None

# 将 four_bits（4位二进制）当作 0 到 1 之间的小数
decimal_value = int(four_bits, 2) / 16  # 将4位二进制转为0到1之间的小数
fsm = NonlinearFSM()
initial_state = decimal_value  # 使用小数值作为初始状态
results_float, period_length = fsm.iterate_and_find_period(initial_state, 2048)
def threshold_results(results):
    return [1 if x > 0.5 else 0 for x in results]
thresholded_results = threshold_results(results_float)
def convert_to_decimal(thresholded_results):
    """
    将 thresholded_results 中相邻的两个二进制值转换为十进制，生成新的序列
    :param thresholded_results: 输入的二进制序列（每个元素为 0 或 1）
    :return: 处理后的十进制序列，长度为 1024，每个元素范围为 0~3
    """
    decimal_results = []
    for i in range(0, len(thresholded_results) // 2):  # 每次取两个相邻值
        # 取两个相邻的值并转换为二进制
        bin_value = thresholded_results[i] * 2  + thresholded_results[i + len(thresholded_results) // 2]  # 转换为二进制表示
        decimal_results.append(bin_value)  # 转换为十进制并保存
    
    return decimal_results

# 示例
decimal_results = convert_to_decimal(thresholded_results)
def accept(accept_set=None, original_list=None):
    return [accept_set[element] for element in original_list]
results = accept([1, 2, 4, 8], decimal_results)
print(results)  
print("--------------------------------")



def detect_period(sequence):
    """
    检测序列是否存在周期，若存在周期，返回周期长度和周期内的数字，否则返回不存在周期。
    :param sequence: 输入的数字序列
    :return: (周期开始位置, 周期长度, 周期内的数字) 或 "No Period" 
    """
    n = len(sequence)
    for period_length in range(1, n//2 + 1):  # 遍历周期长度，从1到n//2
        for start in range(n - period_length):  # 遍历周期开始的位置
            # 判断是否存在周期
            period = sequence[start:start + period_length]  # 提取周期
            for i in range(start + period_length, n, period_length):  # 从周期之后的位置开始检查
                if sequence[i:i + period_length] != period:  # 如果与周期不一致，则不是周期
                    break
            else:
                # 找到周期，返回周期的开始位置和长度以及周期本身
                return start, period_length, period

    return "No Period"  # 没有周期

# 示例
result = detect_period(decimal_results)

if result == "No Period":
    print(result)
else:
    period_start, period_length, period_sequence = result
    print(f"Detected Period Start: {period_start}")
    print(f"Detected Period Length: {period_length}")
    print(f"Period Sequence: {period_sequence}")




[2, 2, 4, 1, 4, 1, 1, 1, 1, 1, 1, 1, 2, 4, 4, 4, 4, 8, 8, 2, 4, 4, 2, 1, 1, 1, 4, 2, 1, 2, 4, 8, 2, 1, 1, 1, 2, 8, 4, 2, 4, 1, 1, 4, 2, 4, 1, 2, 4, 2, 2, 2, 4, 1, 4, 8, 8, 4, 1, 2, 2, 1, 1, 2, 8, 2, 4, 4, 2, 4, 2, 1, 1, 1, 1, 2, 1, 4, 4, 1, 1, 1, 1, 2, 4, 1, 1, 1, 4, 1, 1, 2, 8, 1, 1, 1, 8, 2, 8, 2, 8, 4, 1, 4, 1, 4, 4, 4, 4, 2, 4, 1, 2, 4, 4, 1, 4, 1, 2, 2, 4, 1, 2, 2, 2, 1, 2, 2, 1, 2, 8, 8, 2, 2, 8, 8, 1, 2, 1, 4, 4, 1, 4, 1, 1, 2, 1, 2, 4, 1, 1, 1, 2, 4, 4, 8, 4, 2, 4, 2, 8, 4, 1, 8, 1, 1, 8, 2, 1, 1, 1, 4, 2, 1, 2, 1, 8, 1, 8, 2, 2, 1, 8, 4, 8, 1, 2, 8, 2, 8, 4, 1, 4, 4, 1, 4, 1, 1, 1, 1, 1, 2, 1, 1, 8, 2, 1, 1, 8, 1, 4, 4, 1, 2, 1, 8, 1, 4, 1, 2, 2, 1, 4, 1, 2, 2, 1, 4, 1, 4, 8, 8, 2, 8, 8, 2, 8, 2, 4, 4, 4, 2, 4, 1, 8, 1, 4, 4, 1, 2, 1, 1, 2, 2, 2, 8, 2, 2, 2, 1, 4, 2, 1, 2, 8, 8, 2, 2, 2, 1, 4, 1, 2, 1, 4, 1, 4, 2, 8, 4, 4, 1, 2, 1, 8, 4, 2, 4, 1, 2, 1, 4, 1, 1, 4, 4, 1, 2, 1, 2, 1, 4, 2, 2, 4, 2, 8, 1, 8, 2, 1, 4, 8, 2, 4, 1, 4, 4, 2, 8, 2, 2, 8, 8, 2, 2, 2, 1, 8, 1, 4, 1, 2, 

In [4]:
from collections import Counter
print(Counter(results))


Counter({1: 342, 2: 280, 4: 250, 8: 152})


In [5]:
print(thresholded_results)
import numpy as np
def save_to_bin(data, filename):
    """
    将二进制数据保存到文件。
    :param data: 二进制数据列表（每个元素为0或1）
    :param filename: 输出的文件名
    """
    # 将列表中的数据转换为一个二进制字符串
    bin_str = ''.join(map(str, data))
    
    # 将二进制字符串转换为字节流
    byte_data = int(bin_str, 2).to_bytes((len(bin_str) + 7) // 8, byteorder='big')
    
    # 保存为二进制文件
    with open(filename, 'wb') as f:
        f.write(byte_data)

# 保存到二进制文件
save_to_bin(thresholded_results, './megabitrand_new.bin')

print("Binary file 'megabitrand_new.bin' saved.")


[0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 

In [23]:
from collections import Counter
print(Counter(decimal_results))
## 尝试多次迭代，然后说一下这里每个值基本都是占了25%出现概率


Counter({3: 289, 2: 261, 1: 256, 0: 218})


In [1]:
import numpy as np

class NonlinearFSM:
    def __init__(self):
        self.phi = (1 + 5 ** 0.5) / 2
        self.e = 2.718281828459045
        self.pi = 3.141592653589793
        self.sqrt2 = 2 ** 0.5

    def next_state(self, current_state):
        # 保持状态值为小数类型
        x = current_state
        logistic = 4 * x * (1 - x)
        tent = min(2 * x, 2 * (1 - x))
        sine = abs(np.sin(self.pi * x))
        cubic = abs((3 * x ** 3 - 2 * x) % 1)
        mixed = (logistic * self.phi + tent * self.e + sine * self.pi + cubic * self.sqrt2) % 1
        disturbed = (mixed + x * self.phi) % 1
        return disturbed  # 返回一个小数状态值

    def iterate_and_find_period(self, initial_state):
        current_state = initial_state
        seen_states = {}
        results = []
        iteration = 0

        while True:  # 无限循环，直到找到周期
            if current_state in seen_states:
                period_length = iteration - seen_states[current_state]
                return results, period_length
            seen_states[current_state] = iteration
            current_state = self.next_state(current_state)
            results.append(current_state)
            iteration += 1

# 示例使用
if __name__ == "__main__":
    fsm = NonlinearFSM()
    initial_state = 0.5  # 设置初始状态
    results_float, period_length = fsm.iterate_and_find_period(initial_state)
    print("结果:", results_float)
    print("周期长度:", period_length)

结果: 